# Python assignment mechanism

Let’s look at an example first:

In [1]:
x = [1, 2, 3]
y = x
x[1] = 100
print(y)

[1, 100, 3]


Changing the value of variable `x` will also change the value of variable `y`. This is related to the internal assignment mechanism of **Python**.

## Simple type

Let’s first look at the execution process of this piece of code in **Python**.

```python
x = 500
y = x
y = 'foo'
```

- `x = 500`

**Python** allocates a `PyInt` size memory `pos1` to store the object `500`. Then, Python lets the variable `x` point to this memory in the namespace. Note that integers are immutable types. , so the contents of this memory are immutable.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable)| `x : pos1` |

- `y = x `

**Python** does not use new memory to store the value of variable `y`, but in the namespace, let variable `y` and variable `x` point to the same memory space.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable)|`x : pos1`<br> `y : pos1`|

- `y = 'foo'`

**Python** now allocates a `PyStr` size memory `pos2` to store the object `foo`, and then changes the object pointed to by the variable `y`.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable)<br> `pos2 : PyStr('foo')` (immutable)|`x : pos1`<br>`y : pos2`|

To verify this process, use the `id` function.

     id(x)

Returns the memory address of variable `x`.

In [2]:
x = 500
id(x)

2267300357584

In [3]:
y = x
id(y)

2267300357584

You can also use `is` to determine whether they point to the same thing:

In [4]:
x is y

True

Now `y` points to another piece of memory:

In [5]:
y = 'foo'
id(y)

2267248647344

In [6]:
x is y

False

**Python** will assign a value to every object that appears, even if their values are the same, for example:

In [7]:
x = 500
id(x)

2267300357328

In [8]:
y = 500
id(y)

2267300356432

In [9]:
x is y

False

However, in order to improve memory utilization efficiency, for some simple objects, such as some int objects with small values, **Python** adopts the method of reusing object memory:

In [10]:
x = 2
id(x)

140715959972680

In [11]:
y = 2
id(y)

140715959972680

In [12]:
x is y

True

## Container type

Now look at another piece of code:

```python
x = [500, 501, 502]
y = x
y[1] = 600
y = [700, 800]
```

- `x = [500, 501, 502]`

Python allocates memory `pos1`, `pos2`, `pos3` (immutable) for 3 PyInts, then allocates a section of memory `pos4` for the list, which contains 3 locations, pointing to these 3 memories respectively, and finally lets the variables `x` points to this list.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable) <br> `pos2 : PyInt(501)` (immutable) <br>`pos3 : PyInt(502)` (immutable) <br> `pos4 : PyList (pos1, pos2, pos3)` (variable)|`x : pos4`|

- `y = x`

No new object is created, just point `y` to `pos4`.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable) <br> `pos2 : PyInt(501)` (immutable) <br> `pos3 : PyInt(502)` (immutable) <br> `pos4 : PyList (pos1, pos2, pos3)` (variable)|`x : pos4`<br>`y : pos4`|

- `y[1] = 600`

It turns out that the location `y[1]` points to `pos2`. Since the value of `pos2` cannot be modified, new memory `pos5` is allocated for `600` first.

Then change the position pointed by `y[1]` to `pos5`. At this time, since the object at `pos2` is no longer useful, **Python** will automatically call the garbage disposal mechanism to recycle it.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable) <br> `pos2 :` Garbage collection<br> `pos3 : PyInt(502)` (immutable) <br> `pos4 : PyList(pos1, pos5, pos3 )` (variable)<br>`pos5 : PyInt(600)` (immutable) |`x : pos4`<br> `y : pos4`|

- `y = [700, 800]`

First create this list and then point the variable `y` to it.

|memory|namespace|
|---|---|
|`pos1 : PyInt(500)` (immutable) <br> `pos3 : PyInt(502)` (immutable) <br>`pos4 : PyList(pos1, pos5, pos3)` (variable)<br> `pos5 : PyInt(600)` (immutable) <br>`pos6 : PyInt(700)` (immutable)<br>`pos7 : PyInt(800)` (immutable)<br>`pos8 : PyList( pos6, pos7)` (variable)|`x : pos4` <br> `y : pos8`|

To verify this process:

In [13]:
x = [500, 501, 502]
print(id(x[0]))
print(id(x[1]))
print(id(x[2]))
print(id(x))

2267300357904
2267300357328
2267300356720
2267302596928


Assignment, `id(y)` is the same as `id(x)`.

In [14]:
y = x
print(id(y))

2267302596928


In [15]:
x is y

True

Modify `y[1]`, `id(y)` does not change.

In [16]:
y[1] = 600
print(id(y))

2267302596928


The values of `id(x[1])` and `id(y[1])` changed.

In [17]:
print(id(x[1]))
print(id(y[1]))

2267302271376
2267302271376


Change the value of `y`, the value of `id(y)` changes

In [18]:
y = [700, 800]
print(id(y))
print(id(x))

2267301993216
2267302596928
